In [1]:
!pip install dlib face_recognition opencv-python numpy

  Using cached dlib-20.0.0.tar.gz (3.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached face_recognition-1.3.0-py2.py3-none-any.whl.metadata (21 kB)
  Using cached face_recognition_models-0.3.0-py2.py3-none-any.whl
Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)
  Created wheel for dlib: filename=dlib-20.0.0-cp312-cp312-win_amd64.whl size=2963012 sha256=f0801015eab6f536719a3e39ce6d90975122e92072a03fc888bd7f0e7c2eca69
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\35\bc\f4\3551aa7a295bf59b6cbf9cb588197b668052e5ec92a02aff7f
Successfully built dlib


In [3]:
import face_recognition
import cv2
import numpy as np
import os
from datetime import datetime
import csv



ruta_dataset = 'dataset'
rostros_codificados = []
nombres = []


for archivo in os.listdir(ruta_dataset):
    if archivo.endswith('.jpg'):
        imagen = face_recognition.load_image_file(os.path.join(ruta_dataset, archivo))
        codificacion = face_recognition.face_encodings(imagen)

        if codificacion:  
            rostros_codificados.append(codificacion[0])
            nombre = os.path.splitext(archivo)[0]  
            nombres.append(nombre)
        else:
            print(f" No se detectó rostro en: {archivo}")



if not os.path.exists("registro_de_asistencias"):
    os.makedirs("registro_de_asistencias")


fecha_hoy = datetime.now().strftime("%Y-%m-%d")
archivo_asistencia = f"registro_de_asistencias/{fecha_hoy}.csv"


if not os.path.exists(archivo_asistencia):
    with open(archivo_asistencia, "w", newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Nombre", "Fecha y hora"])


asistencias_hoy = set()


cam = cv2.VideoCapture(0)
print(" Cámara encendida. Presiona 'q' para salir.")

while True:
    ret, frame = cam.read()
    if not ret:
        break

    
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    
    ubicaciones = face_recognition.face_locations(rgb_frame)
    codificaciones = face_recognition.face_encodings(rgb_frame, ubicaciones)

    for codificacion, ubicacion in zip(codificaciones, ubicaciones):
        
        distancias = face_recognition.face_distance(rostros_codificados, codificacion)
        mejor_match = np.argmin(distancias)

        if distancias[mejor_match] < 0.6:
            nombre_detectado = nombres[mejor_match]

            
            if nombre_detectado not in asistencias_hoy:
                asistencias_hoy.add(nombre_detectado)
                hora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                print(f" Asistencia registrada: {nombre_detectado} a las {hora}")
                
                with open(archivo_asistencia, "a", newline='') as f:
                    writer = csv.writer(f)
                    writer.writerow([nombre_detectado, hora])

            
            top, right, bottom, left = ubicacion
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, nombre_detectado, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        else:
            # Desconocido
            top, right, bottom, left = ubicacion
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            cv2.putText(frame, "Desconocido", (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    cv2.imshow("Reconocimiento Facial", frame)

    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cam.release()
cv2.destroyAllWindows()
print(" Reconocimiento finalizado.")


📷 Cámara encendida. Presiona 'q' para salir.
✅ Asistencia registrada: dilan a las 2025-06-28 23:51:07
👋 Reconocimiento finalizado.
